<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/002_Embeddings1_Portal_da_Transpar%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EM CONSTRUÇÃO

# Experimento 002
##Objetivo: Experimentos com word embeddings usando como corpo o campo de descrição do dataset público do portal da transparência.
###Descrição: Experimentos iniciais para criação e visualização de word embeddings

Dica: No COLAB Use CTRL SPACE ao invés de TAB para "autocompletar". Ex:pd.re [CTRL SPACE] vai mostrar uma lista contendo as funções e atributos que começam com pd.re (como read_csv, ...) 

In [1]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pprint

#1 - Carga do dataset

In [2]:
#Copia os dados das NFEs do portal da cidadância pro drive virtual.
import requests  
file_url = "https://raw.githubusercontent.com//mfilipak/AFRAC_IA/main/DATASET/202201_NFe_NotaFiscalItem.zip"
r = requests.get(file_url, stream = True) 

with open("portal.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)

DATA_FILE = "portal.zip"
df = pd.read_csv(DATA_FILE, encoding="CP1252",sep=";")
print("O dataframe completo contém:",len(df),"linhas")

O dataframe completo contém: 324056 linhas


In [3]:
df3 = df[['DATA EMISSÃO','DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'CFOP']]
df3.columns = ["DATA", "DESCR", "NCM", "CFOP"]
df3 = df3[df3["NCM"]!=-1] #Filtrando NCMs = -1

text_lengths = np.array([len(_) for _ in df3['DESCR']])
df3 = df3[text_lengths>=3]

#Caso queira eliminar as repetições rodar a linha abaixo
df3 = df3.drop_duplicates(subset=["DESCR"])

#2 - Carga do text corpus

In [29]:
#No jargão do doc2vec, cada descrição é um document e o conjunto de docs um corpus
#Link: https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#sphx-glr-auto-examples-core-run-core-concepts-py
random.seed(42)
#text_corpus = random.sample(list(df3["DESCR"]), 100)
text_corpus = random.sample(list(df3["DESCR"][:30]), 10)
text_corpus
#Deixar bem curtinho o corpo pra ir arrumando o código e testando os resultados. Pra sequência abaixo fazer mais sentido é bom que tenha alguma repetição de palavras

['GASOLINA COMUM',
 'MLTD203UXAZ CARTUCHO DE TONER PRETO 15K PAGINAS',
 'HP RESERVATORIO DE RESIDUO DE TONER',
 'UVA ITALIA',
 'CLTC603LXAZ CARTUCHO DE TONER CIANO 10K PAGINAS',
 'CARTUCHO DE TONER AMARELO 3.5K PAGINAS',
 'CENOURA',
 'CLTC506LXAZ CARTUCHO DE TONER CIANO 3.5K PAGINAS',
 'PLACA LED ILUMINACAO',
 'OLEO DIESEL B S10 ADITIVADO GRID']

#3 - Determinando as palavras muito frequentes 

In [30]:
all_words = []
for d in list(df3["DESCR"]):
    all_words += d.split()
words_counts = pd.DataFrame(all_words).value_counts()
print("Palavras mais frequentemente encontradas")
print(" ".join([_[0] for _ in words_counts.index[:30]]))

Palavras mais frequentemente encontradas
- DE E PARA X COM 1 EM TIPO de A | DO FILTRO C/ DA MM 2 / PARAFUSO CABO Lote: P/ OLEO 100 Ed O MATERIAL 10 KG


#4 - Seguindo com o tutorial do doc2vec

In [31]:
stoplist = set('- DE E PARA X COM EM TIPO de A | DO C/ DA / P/ Ed O'.split(' '))

In [32]:
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
#Ignorei esse filtro pois com poucos exemplos iria cortar quase tudo
processed_corpus = [[token for token in text if frequency[token] > 0] for text in texts]
pprint.pprint(processed_corpus)

[['gasolina', 'comum'],
 ['mltd203uxaz', 'cartucho', 'toner', 'preto', '15k', 'paginas'],
 ['hp', 'reservatorio', 'residuo', 'toner'],
 ['uva', 'italia'],
 ['cltc603lxaz', 'cartucho', 'toner', 'ciano', '10k', 'paginas'],
 ['cartucho', 'toner', 'amarelo', '3.5k', 'paginas'],
 ['cenoura'],
 ['cltc506lxaz', 'cartucho', 'toner', 'ciano', '3.5k', 'paginas'],
 ['placa', 'led', 'iluminacao'],
 ['oleo', 'diesel', 'b', 's10', 'aditivado', 'grid']]


In [33]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(29 unique tokens: ['comum', 'gasolina', '15k', 'cartucho', 'mltd203uxaz']...)


In [25]:
pprint.pprint(dictionary.token2id)

{'10k': 13,
 '15k': 2,
 '3.5k': 16,
 'aditivado': 23,
 'amarelo': 17,
 'b': 24,
 'cartucho': 3,
 'cenoura': 18,
 'ciano': 14,
 'cltc506lxaz': 19,
 'cltc603lxaz': 15,
 'comum': 0,
 'diesel': 25,
 'gasolina': 1,
 'grid': 26,
 'hp': 8,
 'iluminacao': 20,
 'italia': 11,
 'led': 21,
 'mltd203uxaz': 4,
 'oleo': 27,
 'paginas': 5,
 'placa': 22,
 'preto': 6,
 'reservatorio': 9,
 'residuo': 10,
 's10': 28,
 'toner': 7,
 'uva': 12}


In [26]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1)],
 [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(7, 1), (8, 1), (9, 1), (10, 1)],
 [(11, 1), (12, 1)],
 [(3, 1), (5, 1), (7, 1), (13, 1), (14, 1), (15, 1)],
 [(3, 1), (5, 1), (7, 1), (16, 1), (17, 1)],
 [(18, 1)],
 [(3, 1), (5, 1), (7, 1), (14, 1), (16, 1), (19, 1)],
 [(20, 1), (21, 1), (22, 1)],
 [(23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]]
